# Cleaning item-specific data

In [136]:
import pandas as pd
from sqlalchemy import create_engine


driver = 'mysql+pymysql:'
user = 'adria'
password = '00000'
ip = '35.187.114.125'
database = 'vimet'

connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

In [137]:
items = pd.read_sql('items', con = engine)

In [138]:
items.head()

index   Name  Lineitem quantity                            Lineitem name  \
0      0  #1248                  4          Plátano Canarias - 3/4 unidades   
1      1  #1248                  2       Fresón Maresme - 1 caja (500 grs.)   
2      2  #1248                  1                  Tomate Cherry - 200 grs   
3      3  #1248                  1  Nueces California sin cáscara - 200 grs   
4      4  #1248                  3                Mandarina Orri - 500 grs.   

   Lineitem price Lineitem fulfillment status Cancelled at  Refunded Amount  \
0            1.95                   fulfilled         None              0.0   
1            3.50                   fulfilled         None              NaN   
2            1.30                   fulfilled         None              NaN   
3            3.44                   fulfilled         None              NaN   
4            2.00                   fulfilled         None              NaN   

                      Vendor  Tags  Lineitem discount  
0  Fruites i Verdures Rovira  None                  0  
1  Fruites i Verdures Rovira  None                  0  
2  Fruites i Verdures Rovira  None                  0  
3        Llegums Porta Novau  None                  0  
4  Fruites i Verdures Rovira  None                  0

We'll drop the index column as it is redundant and the 'id' (which corresponds to the order and it's redundant with 'Name'

In [139]:
items = items.drop(columns='index')

In [140]:
items.head()

Name  Lineitem quantity                            Lineitem name  \
0  #1248                  4          Plátano Canarias - 3/4 unidades   
1  #1248                  2       Fresón Maresme - 1 caja (500 grs.)   
2  #1248                  1                  Tomate Cherry - 200 grs   
3  #1248                  1  Nueces California sin cáscara - 200 grs   
4  #1248                  3                Mandarina Orri - 500 grs.   

   Lineitem price Lineitem fulfillment status Cancelled at  Refunded Amount  \
0            1.95                   fulfilled         None              0.0   
1            3.50                   fulfilled         None              NaN   
2            1.30                   fulfilled         None              NaN   
3            3.44                   fulfilled         None              NaN   
4            2.00                   fulfilled         None              NaN   

                      Vendor  Tags  Lineitem discount  
0  Fruites i Verdures Rovira  None                  0  
1  Fruites i Verdures Rovira  None                  0  
2  Fruites i Verdures Rovira  None                  0  
3        Llegums Porta Novau  None                  0  
4  Fruites i Verdures Rovira  None                  0

## NaN values

In [141]:
items.isna().sum()

Name                              0
Lineitem quantity                 0
Lineitem name                     0
Lineitem price                    0
Lineitem fulfillment status       0
Cancelled at                   3052
Refunded Amount                2874
Vendor                            1
Tags                           3094
Lineitem discount                 0
dtype: int64

In [142]:
items.shape

(3120, 10)

We will drop 'Tags' as it does not provide any information

In [143]:
items = items.drop(columns='Tags')

We will fill the empty refunded amounts with 0. And Cancelled at with 'not cancelled'

In [144]:
items['Cancelled at'] =items['Cancelled at'].fillna('not cancelled')

In [145]:
items['Refunded Amount'] =items['Refunded Amount'].fillna(0)

There is only one Nan value in vendor. We check what's going on.

In [146]:
items[items['Vendor'].isna()]

Name  Lineitem quantity               Lineitem name  Lineitem price  \
976  #1201                  1  Pechuga de pollo fileteada            3.15   

    Lineitem fulfillment status   Cancelled at  Refunded Amount Vendor  \
976                   fulfilled  not cancelled              0.0   None   

     Lineitem discount  
976                  0

In [147]:
items[items['Lineitem name'].map(lambda x: 'pollo' in x)]

Name  Lineitem quantity  \
24    #1247                  1   
48    #1246                  1   
69    #1245                  1   
95    #1244                  1   
141   #1240                  1   
182   #1239                  1   
197   #1238                  1   
220   #1237                  1   
238   #1236                  1   
255   #1235                  1   
276   #1235                  1   
308   #1234                  1   
318   #1233                  1   
339   #1233                  1   
353   #1232                  1   
373   #1231                  1   
405   #1229                  1   
420   #1227                  1   
433   #1227                  2   
459   #1226                  1   
465   #1226                  1   
479   #1225                  1   
512   #1222                  1   
552   #1220                  1   
565   #1220                  2   
650   #1216                  1   
694   #1215                  1   
738   #1212                  1   
749   #1212                  1   
750   #1212                  1   
...     ...                ...   
2733  #1057                  1   
2751  #1056                  1   
2754  #1055                  2   
2791  #1052                  1   
2792  #1052                  1   
2804  #1050                  1   
2839  #1049                  2   
2840  #1049                  3   
2845  #1048                  2   
2846  #1048                  1   
2854  #1047                  1   
2855  #1047                  1   
2867  #1045                  1   
2868  #1045                  1   
2884  #1044                  1   
2885  #1044                  1   
2909  #1040                  2   
2910  #1040                  3   
2913  #1038                  1   
2932  #1034                  2   
2933  #1034                  1   
2948  #1032                  1   
2949  #1032                  1   
2950  #1032                  1   
2968  #1028                  1   
3034  #1020                  2   
3052  #1019                  1   
3069  #1012                  1   
3071  #1012                  1   
3100  #1009                  1   

                                          Lineitem name  Lineitem price  \
24                Pechuga de pollo fileteada (350 grs.)            3.15   
48      Hamburguesas Gourmet de pollo lisa - 3 unidades            4.80   
69    Brochetas de pechuga de pollo a las finas hier...            4.40   
95                Pechuga de pollo fileteada (350 grs.)            3.15   
141               Pechuga de pollo fileteada (350 grs.)            3.15   
182               Pechuga de pollo fileteada (350 grs.)            3.15   
197               Pechuga de pollo fileteada (350 grs.)            3.15   
220               Pechuga de pollo fileteada (350 grs.)            3.15   
238               Pechuga de pollo fileteada (350 grs.)            3.15   
255               Pechuga de pollo fileteada (350 grs.)            3.15   
276     Hamburguesas Gourmet de pollo lisa - 3 unidades            4.80   
308               Pechuga de pollo fileteada (350 grs.)            3.15   
318               Pechuga de pollo fileteada (350 grs.)            3.15   
339     Hamburguesas Gourmet de pollo lisa - 3 unidades            4.80   
353               Pechuga de pollo fileteada (350 grs.)            3.15   
373               Pechuga de pollo fileteada (350 grs.)            3.15   
405               Pechuga de pollo fileteada (350 grs.)            3.15   
420               Pechuga de pollo fileteada (350 grs.)            3.15   
433     Hamburguesas Gourmet de pollo lisa - 2 unidades            3.20   
459               Pechuga de pollo fileteada (350 grs.)            3.15   
465     Hamburguesas Gourmet de pollo lisa - 2 unidades            3.20   
479               Pechuga de pollo fileteada (350 grs.)            3.15   
512               Pechuga de pollo fileteada (350 grs.)            3.15   
552               Pechuga de pollo fileteada (350 grs.)            3.15   
565     Hamburguesas Go

We can see that the usual vendor for this type of items is 'El Pagés' therefore, we will fill the Nan with this value

In [148]:
items['Vendor'] = items['Vendor'].fillna('El Pagés')

## Variable types

In [149]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3120 entries, 0 to 3119
Data columns (total 9 columns):
Name                           3120 non-null object
Lineitem quantity              3120 non-null int64
Lineitem name                  3120 non-null object
Lineitem price                 3120 non-null float64
Lineitem fulfillment status    3120 non-null object
Cancelled at                   3120 non-null object
Refunded Amount                3120 non-null float64
Vendor                         3120 non-null object
Lineitem discount              3120 non-null int64
dtypes: float64(2), int64(2), object(5)
memory usage: 219.5+ KB


Types seem ok

## Feature values

In [150]:
items.columns

Index(['Name', 'Lineitem quantity', 'Lineitem name', 'Lineitem price',
       'Lineitem fulfillment status', 'Cancelled at', 'Refunded Amount',
       'Vendor', 'Lineitem discount'],
      dtype='object')

#### Quantity -- OK

In [151]:
items['Lineitem quantity'].value_counts()

1     2793
2      250
3       40
4       22
6        4
10       3
16       2
12       2
8        2
5        1
30       1
Name: Lineitem quantity, dtype: int64

Looks good

#### Lineitem name'

In [152]:
items['Lineitem name'].value_counts()

Huevos - 1/2 docena                                     41
Brócoli - 1 unidad                                      40
Pechuga pollo fileteada - 1 pechuga (300 gr)            39
Pera Conference - 4 unidades                            39
Plátano Canarias Verde - 3/4 unidades                   38
Lote de productos básicos - Donativo Banc d'Aliments    38
Zanahorias - medio kilo                                 36
Plátano canario (verde) - 0.5 kg. (3-4 unidades)        35
Almendra Cruda - 125 grs.                               33
Huevos ecológicos (6 unidades)                          33
Manzana Golden - 4 unidades                             33
Nueces California sin cáscara - 125 grs.                32
Calabacín - 500 grs.                                    29
Tomate Monterosa - 500 grs.                             28
Cebolla Reca para guisar (1 Kg. = 3/4 uds.)             27
Calabacines - 500 grs. (2-3 unidades)                   26
Fresón Maresme - 1 caja (500 grs.)                      

After analysing all the patterns we could find, we simplify the description of each item and create a tag list. Below, there's an example of the serch for patterns. We will search for 'pollo' and then the other registers will be changed to others as there is no item 'fruta'

In [153]:
items['Lineitem name'][items['Lineitem name'].map(lambda x: x.startswith('Brocheta'))]

69      Brochetas de pechuga de pollo a las finas hier...
998     Brochetas de pechuga de pollo barbacoa - 5 uni...
1010    Brochetas de pechuga de pollo barbacoa - 5 uni...
2519                       Brocheta de fruta - 3 unidades
Name: Lineitem name, dtype: object

In [154]:
items['Lineitem name'][items['Lineitem name'].map(lambda x: x.startswith('Virutas'))]

1043    Virutas de Jamón de bellota - 150 grs.
Name: Lineitem name, dtype: object

This contains too much information, we will split the content and create tags. The rest of information will go in 'Notes'.

In [161]:
# We will reduce the information contained in the item description by creating a tag with the main food item.

# Create a list with all the items but reducing the ones with 'pollo' and 'pavo' as they are quite recurent and varied
mylst = list(items['Lineitem name'].map(lambda x: 'Pollo' if x.find('pollo') > 0 else ('Pavo' if x.find('pavo') > 0 else x)))

# We now take the fist word of the list as it will describe the main food item for 95% of the items.
tags = []
for tag in mylst:
    tags.append(tag.split(' ')[0])

# Now we solve the special cases
for index, tag in enumerate(tags):
    if tag == 'Cabezas':
        tags[index] = 'Ajo'
    elif tag == 'Cabeza':
        tags[index] = 'Ajo'        
    elif tag == 'Rulo':
        tags[index] = 'Queso'
    elif tag == 'Hamburguesa':
        tags[index] = 'Ternera'
    elif tag == 'Manchego':
        tags[index] = 'Queso'
    elif tag == 'Medio':
        tags[index] = 'Conejo'
    elif tag == 'Brie':
        tags[index] = 'Queso'
    elif tag == 'Emmental':
        tags[index] = 'Queso'
    elif tag == 'Cammenbert':
        tags[index] = 'Queso'
    elif tag == 'Comté':
        tags[index] = 'Queso'
    elif tag == 'Colitas':
        tags[index] = 'Rape'
    elif tag == 'Copos':
        tags[index] = 'Avena'
    elif tag == 'Tapa':
        tags[index] = 'Ternera'
    elif tag == 'Producto':
        tags[index] = 'Others'
    elif tag == 'Picos':
        tags[index] = 'Pan'
    elif tag == 'Tostaditas':
        tags[index] = 'Pan'
    elif tag == 'Barra':
        tags[index] = 'Pan'
    elif tag == '1':
        tags[index] = 'Pan'
    elif tag == 'Alcachofas':
        tags[index] = 'Alcachofa'
    elif tag == 'Azuqui':
        tags[index] = 'Azuki'
    elif tag == 'Baguet':
        tags[index] = 'Pan'
    elif tag == 'Berenjenas':
        tags[index] = 'Berenjena'
    elif tag == 'Broquil':
        tags[index] = 'Brócoli'
    elif tag == 'Calabacines':
        tags[index] = 'Calabacín'
    elif tag == 'Calamares':
        tags[index] = 'Calamar'
    elif tag == 'Ciruelas':
        tags[index] = 'Ciruela'
    elif tag == 'Codornices':
        tags[index] = 'Codorniz'
    elif tag == 'Coles':
        tags[index] = 'Col'
    elif tag == 'Conill':
        tags[index] = 'Conejo'
    elif tag == 'Cous':
        tags[index] = 'Cous-cous'
    elif tag == 'Esparragos':
        tags[index] = 'Esparrago'
    elif tag == 'Feta':
        tags[index] = 'Queso'
    elif tag == 'Filetes':
        tags[index] = 'Filet'
    elif tag == 'Fresones':
        tags[index] = 'Fresón'
    elif tag == 'Garbanzos':
        tags[index] = 'Garbanzo'
    elif tag == 'Idiazábal':
        tags[index] = 'Queso'
    elif tag == 'Judías':
        tags[index] = 'Judía'
    elif tag == 'Lentejas':
        tags[index] = 'Lenteja'
    elif tag == 'Libritos':
        tags[index] = 'Librito'
    elif tag == 'Manzanas':
        tags[index] = 'Manzana'
    elif tag == 'Mejillones':
        tags[index] = 'Mejillón'
    elif tag == 'Naranjas':
        tags[index] = 'Naranja'
    elif tag == 'Nueces':
        tags[index] = 'Nuez'
    elif tag == 'Pasas':
        tags[index] = 'Pasa'
    elif tag == 'Pimientos':
        tags[index] = 'Pimiento'
    elif tag == 'Puerros':
        tags[index] = 'Puerro'
    elif tag == 'Salmon':
        tags[index] = 'Salmón'
    elif tag == 'Tapaplana':
        tags[index] = 'Ternera'
    elif tag == 'Tomates':
        tags[index] = 'Tomate'
    elif tag == 'Zanahorias':
        tags[index] = 'Zanahoria'
    elif tag == 'Boloñesa':
        tags[index] = 'Macarrones'
    elif tag == 'Brocheta':
        tags[index] = 'Others'
    elif tag == 'Cabra':
        tags[index] = 'Queso'
    elif tag == 'Caprice':
        tags[index] = 'Queso'
    elif tag == 'Carne':
        tags[index] = 'Ternera'
    elif tag == 'Cola':
        tags[index] = 'Rape'
    elif tag == 'Fiambre':
        tags[index] = 'Pavo'
    elif tag == 'Flores':
        tags[index] = 'Queso'
    elif tag == 'Hamburg.':
        tags[index] = 'Others'
    elif tag == 'Hamburguesas':
        tags[index] = 'Ternera'
    elif tag == 'Lomo':
        tags[index] = 'Cerdo'
    elif tag == 'Lomos':
        tags[index] = 'Conejo'
    elif tag == 'Mediana':
        tags[index] = 'Conejo'
    elif tag == 'Mini-hamburguesas':
        tags[index] = 'Ternera'
    elif tag == 'Mini-harmburguesas':
        tags[index] = 'Ternera'
    elif tag == 'Zanahorias':
        tags[index] = 'Zanahoria'
    elif tag == 'Muslitos':
        tags[index] = 'Pollo'
    elif tag == 'Muslos':
        tags[index] = 'Pollo'
    elif tag == 'Oveja':
        tags[index] = 'Queso'
    elif tag == 'Pechuga':
        tags[index] = 'Pollo'
    elif tag == 'Pechugas':
        tags[index] = 'Pollo'
    elif tag == 'Tacos':
        tags[index] = 'Cerdo'
    elif tag == 'Tall':
        tags[index] = 'Ternera'
    elif tag == 'Bandeja':
        tags[index] = 'Pollo'
    elif tag == 'Chuleta':
        tags[index] = 'Ternera'
    elif tag == 'Culata':
        tags[index] = 'Ternera'
    elif tag == 'Espalda':
        tags[index] = 'Conejo'
    elif tag == 'Lote':
        tags[index] = 'Others'
    elif tag == 'Mitjana':
        tags[index] = 'Ternera'
    elif tag == 'Pepinos':
        tags[index] = 'Pepino'
    elif tag == 'Virutas':
        tags[index] = 'Jamón'
    elif tag == 'Ajos':
        tags[index] = 'Ajo'
    else:
        pass     

In [162]:
items['tags'] = tags

In [163]:
sorted(items['tags'].unique())

['Aceite',
 'Acelgas',
 'Agua',
 'Aguacate',
 'Ajo',
 'Albaricoque',
 'Albóndigas',
 'Alcachofa',
 'Almejas',
 'Almendra',
 'Alubia',
 'Anacardo',
 'Apio',
 'Arroz',
 'Arándanos',
 'Atun',
 'Avellana',
 'Avena',
 'Azuki',
 'Bananas',
 'Berberechos',
 'Berenjena',
 'Bonito',
 'Boquerones',
 'Brócoli',
 'Bull',
 'Butifarra',
 'Caballa',
 'Cacahuetes',
 'Calabacín',
 'Calabaza',
 'Calamar',
 'Caldo',
 'Calçots',
 'Canelones',
 'Canónigos',
 'Cebolla',
 'Cep',
 'Cerdo',
 'Cerezas',
 'Cesta',
 'Champiñón',
 'Chirimoya',
 'Chocolate',
 'Chorizo',
 'Ciruela',
 'Codorniz',
 'Cogollos',
 'Col',
 'Coliflor',
 'Conejo',
 'Confit',
 'Costillas',
 'Cous-cous',
 'Croquetas',
 'Dorada',
 'Dátiles',
 'Endivias',
 'Eneldo',
 'Ensalada',
 'Entrecot',
 'Escarola',
 'Espaguetis',
 'Esparrago',
 'Espectec',
 'Espinacas',
 'Filet',
 'Frambuesas',
 'Fresón',
 'Frijoles',
 'Fuet',
 'Gamba',
 'Garbanzo',
 'Girgola',
 'Granada',
 'Granola',
 'Higos',
 'Huevos',
 'Hummus',
 'Jamón',
 'Judía',
 'Kakis',
 'Kiwi',


In [23]:
items.groupby('tags').count()

Name  Lineitem quantity  Lineitem name  Lineitem price  \
tags                                                                  
1               1                  1              1               1   
Aceite          8                  8              8               8   
Acelgas        28                 28             28              28   
Agua            2                  2              2               2   
Aguacate       45                 45             45              45   
Ajos            4                  4              4               4   
Albaricoque    10                 10             10              10   
Albóndigas     27                 27             27              27   
Alcachofa       3                  3              3               3   
Alcachofas      8                  8              8               8   
Almejas         8                  8              8               8   
Almendra       37                 37             37              37   
Alubia         26                 26             26              26   
Anacardo        8                  8              8               8   
Apio            6                  6              6               6   
Arroz          23                 23             23              23   
Arándanos      12                 12             12              12   
Atun            6                  6              6               6   
Avellana       20                 20             20              20   
Azuki           1                  1              1               1   
Azuqui          7                  7              7               7   
Baguet          1                  1              1               1   
Bananas        10                 10             10              10   
Bandeja        11                 11             11              11   
Barra           4                  4              4               4   
Berberechos     6                  6              6               6   
Berenjena      13                 13             13              13   
Berenjenas     18                 18             18              18   
Boloñesa        3                  3              3               3   
Bonito          1                  1              1               1   
...           ...                ...            ...             ...   
Rúcula         13                 13             13              13   
Salchichas     15                 15             15              15   
Salmon         13                 13             13              13   
Salmonetes      8                  8              8               8   
Salmón         14                 14             14              14   
Salsa           2                  2              2               2   
Sandía          5                  5              5               5   
Sardina         3                  3              3               3   
Sepia          13                 13             13              13   
Shiitake        3                  3              3               3   
Sobrasada       2                  2              2               2   
Soja            2                  2              2               2   
Solomillo       7                  7              7               7   
Spaghetti       9                  9              9               9   
Sumaia          1                  1              1               1   
Tacos           1                  1              1               1   
Tall           19                 19             19              19   
Tapa            3                  3              3               3   
Tapaplana       3                  3              3               3   
Tomate        119                119            119             119   
Tomates        16                 16             16              16   
Tortilla        9                  9              9               9   
Tostaditas      3                  3              3               3   
Uva             4                  4 

#### 'Lineitem price' --OK

Looks good

#### 'Lineitem requires shipping' -- OK

In [ ]:
items['Lineitem requires shipping'].value_counts()

####  'Lineitem taxable' -- OK

In [ ]:
items['Lineitem taxable'].value_counts()

#### 'Lineitem fulfillment status' -- OK

In [ ]:
items['Lineitem fulfillment status'].value_counts()

# Load data

In [ ]:
from sqlalchemy import create_engine


driver = 'mysql+pymysql:'
user = 'adria'
password = '00000'
ip = '35.187.114.125'
database = 'vimet'

connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

In [ ]:
items.to_sql('items', con = engine, if_exists='replace')